<a href="https://colab.research.google.com/github/glitchheadgit/smilesChessFraud/blob/main/chessFraud_Egor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs and imports

In [1]:
!pip install transformers datasets bertviz -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 319.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requi

In [2]:
import pandas as pd
import numpy as np

import torch
from tqdm.auto import tqdm, trange

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizerFast
from transformers import DataCollatorWithPadding
from tokenizers import BertWordPieceTokenizer
from torch.utils.data import DataLoader
from datasets import Dataset
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

AttributeError: module 'pyarrow.lib' has no attribute 'ListViewType'

# Datasets

In [19]:
ds = pd.read_parquet("part-00000.parquet")
temp_array = ds["Moves_clean"].join(' ')
ds["Moves_clean_str"] = temp_array


# Train tokinizer

Make corpus of moves

In [40]:
moves = []
out_file = open("Moves.txt","w")
for i in ds["Moves_clean"]:
  for j in i:
    moves.append(j)
    print(j,file = out_file,end = ' ')
print(moves)

TypeError: 'str' object cannot be interpreted as an integer

In [41]:
special_tokens = [
  "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"
]

files = ["Moves.txt"]

vocab_size = 30_522

max_length = 300
# whether to truncate
truncate_longer_samples = False

# initialize the WordPiece tokenizer
tokenizer = BertWordPieceTokenizer()
# train the tokenizer
tokenizer.train(files=files, vocab_size=vocab_size, special_tokens=special_tokens)
# enable truncation up to the maximum 512 tokens
tokenizer.enable_truncation(max_length=max_length)

Save tokenizer

In [42]:
import os
import json
model_path = "pretrained-bert"
# make the directory if not already there
if not os.path.isdir(model_path):
  os.mkdir(model_path)
tokenizer.save_model(model_path)



['pretrained-bert/vocab.txt']

In [43]:
with open(os.path.join(model_path, "config.json"), "w") as f:
  tokenizer_cfg = {
      "do_lower_case": True,
      "unk_token": "[UNK]",
      "sep_token": "[SEP]",
      "pad_token": "[PAD]",
      "cls_token": "[CLS]",
      "mask_token": "[MASK]",
      "model_max_length": max_length,
      "max_len": max_length,
  }
  json.dump(tokenizer_cfg, f)

TypeError: 'str' object cannot be interpreted as an integer

In [45]:
Tokenizer = BertTokenizerFast.from_pretrained(model_path)


# Encoding
